In [91]:
import torch
from torch import nn as nn
import torchvision
from torch.utils.data import DataLoader, dataset
from torchvision.transforms import AutoAugment, Normalize, Compose, ToTensor
import torch.nn.functional as F
from torchmetrics import Accuracy

In [131]:
class accuracy:
    def __init__(self):
        pass
    def update(self, x,y):
        pass
    def reset(self):
        pass

In [8]:
def __normalize__(x):
    return x/255.
normalize = torch.jit.script(__normalize__)

In [2]:
from torchvision.datasets import CIFAR100

In [34]:
data = CIFAR100("./data", train = True, download = True, transform = Compose([AutoAugment(),ToTensor(), normalize]))
test_data = CIFAR100("./data", train = False, download = True, transform = Compose([AutoAugment(),ToTensor(),  normalize]))

Files already downloaded and verified
Files already downloaded and verified


In [155]:
train_set = DataLoader(data, 128, shuffle = True, num_workers=2)
test_set = DataLoader(test_data, 128, shuffle = False, num_workers=2)

In [156]:
class VGG5(nn.Module):
    def __init__(self):
        super().__init__()
        self.seq_1 = nn.Sequential(nn.Conv2d(3, 64, 2,1, ), nn.ReLU(),
                                   nn.Conv2d(64, 64, 2,1), nn.ReLU(), 
                                   nn.MaxPool2d(2,2), 
                                   nn.BatchNorm2d(64))
        self.seq_2 = nn.Sequential(nn.Conv2d(64, 128, 2,1, padding = "same"), nn.ReLU(),
                                   nn.Conv2d(128, 128, 2,1, padding = "same"), nn.ReLU(), 
                                   nn.MaxPool2d(2,2), 
                                   nn.BatchNorm2d(128))
        self.seq_3 = nn.Sequential(nn.Conv2d(128, 256, 2,1), nn.ReLU(),
                                   nn.Conv2d(256, 256, 2,1), nn.ReLU(), 
                                   nn.MaxPool2d(2,2), 
                                   nn.BatchNorm2d(256, ))
        self.f = nn.Flatten()
        self.dense = nn.Sequential(nn.Dropout(0.5), nn.Linear(1024, 512), nn.ReLU(), nn.Linear(512, 100))
    def forward(self, x):
        x = self.seq_1(x)
        x = self.seq_2(x)
        x = self.seq_3(x)
        x = self.f(x)
        x = self.dense(x)
        return x
        
        
        

In [157]:
model_ = VGG5()
model = torch.nn.DataParallel(model_)
model.to(0)


In [158]:
def loss(x,y):
    return nn.CrossEntropyLoss()(model(x),y)

In [165]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

In [ ]:
while True:
    L = []
    loss__ = []
    model.train()

    for i, batch in enumerate(train_set):
        x, y = batch[0].cuda(), batch[1].cuda()
    
        loss_ = loss(x,y)
        loss_.backward()
        optimizer.step()
        optimizer.zero_grad()
        loss__.append(loss_.item())

    model.eval()
    with torch.no_grad():
        for j, batch in enumerate(test_set):
            x, y = batch[0].cuda(), batch[1].cuda()
            L.append(sum(torch.argmax(model(x), dim = 1) == y).item())
    print(sum(L)/len(L), sum(loss__)/len(loss__))

54.55696202531646 1.9863350476755206
54.78481012658228 1.8483372271213385
55.822784810126585 1.820088120372704
55.50632911392405 1.8066151419563976
51.51898734177215 1.785859256449258
50.037974683544306 1.7706097894922241
53.164556962025316 1.7643994754537597
52.48101265822785 1.7505749390863092
